In [1]:
import fitz
import requests
import io
import pickle
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import stanza
import json

In [2]:
path = "../data/rawtexts/"
filenames = os.listdir(path)
len(filenames)

11615

In [3]:
filename = filenames[10]
with open(path + filename) as f:
    text = f.read()
print(text[:10000])

I N F O R M A C E . O C Í R K V I 1 9 8 1 Jsem -unaven a za smrt prosľrn Boha - jen nevidět u2, jak j i ita ctnost, a vynášena nicka přeubohá, a křivdju rozšlapána r:./, :.n:i?st, a oaívána zlatem nemc/ic• u; ľ .jt, a v trhu'prodávána g.1v.:í 't, a pokálena' "bezúhonná vrvicnost, a utloukáno to, co eilnti ;j st, a umení jak .paniku v.'Ľ'dô, a doktor Blbec káže ge:i:;vVn, a lumkové'se posmívá.i.f pz-vrdě, a Ľjb;.o babě Zlu jak r.;~icj^-T důta • tím světemounaven, tak rád bych - Leč tebe zůstavit mu na pospas? William Shakespeare [pageend1]- 1 - Dojis Juliu_^abriSovii_tiskupu^v_Trnavět V Praze 6. 2. 1S81. Vaše Bxelence ! Jsem stále žádán věřícími, abych Vám sdělil toto. aši katolíci přijali velmi kladně prohlášení bratislavských bohoslovců o PIT. ITyní se s bolestí dovídáme, že bohoslovcům bylo sděleno, aby ne- nastupovali do bratislavského semináře. Prý budova praskla. Tento důvod sc nám zdá směšný; jde jistě o záminku. 0 jakou, potvrzují výslechy jednotlivých bohoslovců, které provádí 

In [4]:
text.count("[pageend")

10

In [5]:
nlp = stanza.Pipeline("cs")

2025-05-21 12:40:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-21 12:40:39 INFO: Downloaded file to /home/jupyter-vojta/stanza_resources/resources.json
2025-05-21 12:40:39 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_nocharlm |
| depparse  | pdt_nocharlm |

2025-05-21 12:40:39 INFO: Using device: cuda
2025-05-21 12:40:39 INFO: Loading: tokenize
/srv/venvs/default_venv/lib/python3.12/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2025-05-21 12:40:41 INFO: Loading: mwt
2025-05-21 12:40:41 INFO: Loading: pos
2025-05-21 12:40:42 INFO: Loading: lemma
2025-05-21 12:40:43 INFO: Loading: depparse
2025-05-21 12:40:43 INFO: Done loading processors!


In [7]:
%%time
doc = nlp(text)

CPU times: user 4.55 s, sys: 247 ms, total: 4.79 s
Wall time: 2.43 s


In [15]:
lemmatized_sents = []
doc_sents_data = []
for sent in doc.sentences:
    lemmatized_sents.append([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]])
    doc_sents_data.append((sent.text, [(w.text, w.lemma, w.upos, ((w.start_char or 0) - (sent.tokens[0].start_char or 0), (w.end_char or 0) - (sent.tokens[0].start_char or 0))) for w in sent.words]))


print(lemmatized_sents[100:110])

[['chtít', 'maskovat', 'skutečnost'], ['ja', 'ko', 'hrob', 'pěkný', 'zelený', 'travička', 'pach', 'zápach', 'hen', 'loba', 'kost'], ['vést'], ['chtít', 'svoboda', 'svoboda', 'svo', 'bod', 'myšlení', 'slovo'], ['přijít', 'jediný', 'pravý', 'věč', 'ná', 'svoboda', 'bůh', 'popírat', 'svědčit'], ['vynakládat', 'velký', 'úsilí', 'existovat'], ['utvrzovat', 'víra', 'bůh', 'jo', 'říkat', 'náboženství', 'vymyslit', 'otrokář', 'otrok', 'čekat', 'osvoboditel'], ['Á', 'existovat', 'velký', 'otroctví', 'moderní'], ['stát', 'otrok', 'xnusit', 'mluvit', 'psát'], ['nadiktovat']]


In [16]:
doc_sents_data[:10]

[('I N F O R M A C E . O C Í R K V I 1 9 8 1 Jsem -unaven a za smrt prosľrn Boha - jen nevidět u2, jak j i ita ctnost, a vynášena nicka přeubohá, a křivdju rozšlapána r:./, :.n:i?st, a oaívána zlatem nemc/ic• u; ľ .jt, a v trhu\'prodávána g.1v.:í \'t, a pokálena\' "bezúhonná vrvicnost, a utloukáno to, co eilnti ;j st, a umení jak .paniku v.\'Ľ\'dô, a doktor Blbec káže ge:i:;',
  [('I', 'i', 'CCONJ', (0, 1)),
   ('N', 'N', 'NOUN', (2, 3)),
   ('F', 'F', 'NOUN', (4, 5)),
   ('O', 'O', 'NOUN', (6, 7)),
   ('R', 'R', 'NOUN', (8, 9)),
   ('M', 'M', 'NOUN', (10, 11)),
   ('A', 'A', 'NOUN', (12, 13)),
   ('C', 'C', 'NOUN', (14, 15)),
   ('E', 'E', 'NOUN', (16, 17)),
   ('.', '.', 'PUNCT', (18, 19)),
   ('O', 'o', 'ADP', (20, 21)),
   ('C', 'C', 'NOUN', (22, 23)),
   ('Í', '"', 'PUNCT', (24, 25)),
   ('R', 'R', 'NOUN', (26, 27)),
   ('K', 'K', 'NOUN', (28, 29)),
   ('V', 'V', 'NOUN', (30, 31)),
   ('I', 'I', 'NOUN', (32, 33)),
   ('1', '1', 'NUM', (34, 35)),
   ('9', '9', 'NUM', (36, 37)),
   

In [6]:
os.makedirs("../data/lemsents/", exist_ok=True)
os.makedirs("/srv/data/scriptum/sents_data/", exist_ok=True)

In [6]:
processed = []
def get_lemmatized_sentences(filename):
    with open(path + filename, encoding='utf-8', errors='replace') as f:
        text = f.read()
    if filename not in []: # "studie_1990_132_ocr.pdf"]:
        ready = os.listdir("../data/lemsents/")
        if filename not in ready:
            lemmatized_sents = []
            doc_sents_data = []
            try:
                text = re.sub(r"\[pageend\d+\]", " ", text)
                chunks = re.findall(r'.{0,50000}\.\s', text, re.DOTALL)
                chunks_end = sum([len(chunk) for chunk in chunks])
                chunks.append(text[chunks_end:])
                docs = [nlp(chunk) for chunk in chunks]
                for doc in docs:
                    for sent in doc.sentences:
                        lemmatized_sents.append(" ".join([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]]) + "\n")
                        doc_sents_data.append((filename, sent.text, [(w.text, w.lemma, w.upos, ((w.start_char or 0) - (sent.tokens[0].start_char or 0), (w.end_char or 0) - (sent.tokens[0].start_char or 0))) for w in sent.words]))
            except:
                pass
            processed.append(filename)
            pathfn = "../data/lemsents/" + filename
            with open(pathfn, 'w') as f:
                f.writelines(lemmatized_sents)
            jsons_pathfn = "/srv/data/scriptum/sents_data/" + filename.replace(".txt", ".json")
            with open(jsons_pathfn, "w") as f:
                json.dump(doc_sents_data, f)
            if len(ready) in range(0, len(filenames), 100):
                print(len(ready))

In [52]:
filename = filenames[0]
get_lemmatized_sentences(filename)

In [54]:
with open("../data/lemsents/" + filename, "r") as f:
    sents = f.readlines()
sents[:5]

['P b b Wi klub kulturní měsíčník ČECH Slovák Rakousko obsah ČR senát lékař hrozit obrat preference strana SR rok skromnost Slovensko strana Havel mezera zákon prosoukávat podvodník strana\n',
 'prezidentův syn čekat Rakousko strana vyšetřování odsun brněnský Němec skončený strana\n',
 'opožděný návrat dům Kazachstán strana kulturní klub Čech Slovák zvát valný hromada kulturní klub Čech Slovák Rakousko leden středa klubovní hospoda Schlösselgass Vídeň\n',
 'mít přijít strana leden ročník český slovenský Vídeň kulturní klub cech Slovák Rakousko zvát valný hromada leden středa klubovní hospoda Schlösselgass Vídeň\n',
 'vidět schůzování smysluplný trávení čas vyžadovat stanovy najejichž dodržování spolkový policie dohlížet\n']

In [57]:
with open("/srv/data/scriptum/sents_data/" + filename.replace(".txt", ".json"), "r") as f:
    doc_sents_data = json.load(f)
doc_sents_data[:5]

[['kulturni-klub_1996_01-10_ocr.txt',
  'P.b.b. Erscheinungsort und Verlagspostamt — 1090 Wi KLUB 10 KULTURNÍ MĚSÍČNÍK ČECHŮ A SLOVÁKŮ V RAKOUSKU Z OBSAHU: ČR: Kam se senátem, lékaři znovu hrozí, obrat v preferencích strana 3 SR: "Rok skromnosti" pre Slovensko strana 4 Havel: Mezerami v zákonech se prosoukávají podvodníci vzhůru strana 5',
  [['P', 'P', 'NOUN', [0, 1]],
   ['.', '.', 'PUNCT', [1, 2]],
   ['b', 'b', 'NOUN', [2, 3]],
   ['.', '.', 'PUNCT', [3, 4]],
   ['b', 'b', 'NOUN', [4, 5]],
   ['.', '.', 'PUNCT', [5, 6]],
   ['Erscheinungsort', 'Erscheinungsort', 'X', [7, 22]],
   ['und', 'und', 'X', [23, 26]],
   ['Verlagspostamt', 'Verlagspostamt', 'X', [27, 41]],
   ['—', '—', 'PUNCT', [42, 43]],
   ['1090', '1090', 'NUM', [44, 48]],
   ['Wi', 'Wi', 'NOUN', [49, 51]],
   ['KLUB', 'klub', 'NOUN', [52, 56]],
   ['10', '10', 'NUM', [57, 59]],
   ['KULTURNÍ', 'kulturní', 'ADJ', [60, 68]],
   ['MĚSÍČNÍK', 'měsíčník', 'ADJ', [69, 77]],
   ['ČECHŮ', 'ČECH', 'PROPN', [78, 83]],
   ['A', 

In [ ]:
for filename in filenames[6750:]:
    get_lemmatized_sentences(filename)

In [ ]:
print("hello")

In [8]:
len(os.listdir("../data/lemsents/"))

9418

In [10]:
len(os.listdir("/srv/data/scriptum/sents_data/"))

9418